In [16]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

# Load and clean the distance matrix
df = pd.read_excel("Downloads/dairy_farm_travel_times.xlsx", index_col=0)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(1e6, inplace=True)  # Large number to avoid invalid paths
distance_matrix = df.values
n = len(df)

# Create optimization model
m = Model("TSP")

# Decision variables
x = {(i, j): m.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}")
     for i in range(n) for j in range(n) if i != j}

# Subtour elimination variables
u = {i: m.addVar(vtype=GRB.CONTINUOUS, lb=1, ub=n-1, name=f"u_{i}")
     for i in range(1, n)}

# Objective function
m.setObjective(quicksum(distance_matrix[i][j] * x[i, j]
                        for i in range(n) for j in range(n) if i != j), GRB.MINIMIZE)

# Constraints: enter and exit each node exactly once
for i in range(n):
    m.addConstr(quicksum(x[i, j] for j in range(n) if i != j) == 1)
    m.addConstr(quicksum(x[j, i] for j in range(n) if i != j) == 1)

# Subtour elimination (MTZ)
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            m.addConstr(u[i] - u[j] + (n - 1) * x[i, j] <= n - 2)

# Solve the model
m.optimize()

# Extract solution
if m.status == GRB.OPTIMAL:
    solution = [(i, j) for i in range(n) for j in range(n) if i != j and x[i, j].X > 0.5]
    print("Optimal route edges:")
    for edge in solution:
        print(f"{df.index[edge[0]]} → {df.index[edge[1]]}")
    print("Total optimal travel time:", m.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 212 rows, 224 columns and 966 nonzeros
Model fingerprint: 0xfc212ee4
Variable types: 14 continuous, 210 integer (210 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [5e+00, 1e+06]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 1e+01]
Presolve time: 0.01s
Presolved: 212 rows, 224 columns, 1938 nonzeros
Variable types: 14 continuous, 210 integer (210 binary)
Found heuristic solution: objective 7000302.0000

Root relaxation: objective 1.000374e+06, 78 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1000374.0000 1000374.00  0.00%     -    0s

Explored 1 nodes (78 

In [18]:
import pandas as pd
import numpy as np
from python_tsp.heuristics import solve_tsp_local_search

# Load and clean the matrix
df = pd.read_excel("Downloads/dairy_farm_travel_times.xlsx", index_col=0)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(1e6, inplace=True)
distance_matrix = df.values

# Solve using 2-opt heuristic
route, total_cost = solve_tsp_local_search(distance_matrix)

# Print results
print("Route (visit order):")
for idx in route:
    print(df.index[idx], end=" → ")
print(df.index[route[0]])  # to complete the cycle

print("\nHeuristic total travel time:", total_cost)


Route (visit order):
Springfield → Cook → Zakeri → Hunt → Dell → Herman → Thompson → Barstow → Fletcher → Wolfe farm → Wysocki → Hav → Chase → Wilson → Grace Hill → Springfield

Heuristic total travel time: 5000353.0
